<a href="https://colab.research.google.com/github/rsh006/Webpage/blob/master/Udemy_dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install panel

In [ ]:
!pip install hvplot

In [ ]:
!pip install holoviews

In [ ]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')
import hvplot.pandas
import holoviews as hv
hv.extension('bokeh')

In [ ]:
from google.colab import files

In [ ]:
uploaded = files.upload()

In [ ]:
df = pd.read_csv('udemy_courses.csv')

In [ ]:
# List of popular country names
countries = ['USA', 'Canada', 'UK', 'Germany', 'France', 'India', 'Australia', 'Japan', 'Brazil', 'South Africa', 'China', 'Russia', 'Mexico', 'Italy', 'Spain', 'Netherlands', 'Singapore', 'Sweden', 'Norway', 'Denmark', 'Switzerland', 'South Korea', 'New Zealand', 'Ireland', 'Belgium', 'Argentina', 'Chile', 'Portugal', 'Greece']

# Adding a new column 'country' with random country names
df['country'] = np.random.choice(countries, len(df))

In [ ]:
df.head()

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject,country,year
0,1070968,Ultimate Investment Banking Course,https://www.udemy.com/ultimate-investment-bank...,True,200,2147,23,51,All Levels,1.5,2017-01-18 20:58:58+00:00,Business Finance,Denmark,2017
1,1113822,Complete GST Course & Certification - Grow You...,https://www.udemy.com/goods-and-services-tax/,True,75,2792,923,274,All Levels,39.0,2017-03-09 16:34:20+00:00,Business Finance,Canada,2017
2,1006314,Financial Modeling for Business Analysts and C...,https://www.udemy.com/financial-modeling-for-b...,True,45,2174,74,51,Intermediate Level,2.5,2016-12-19 19:26:30+00:00,Business Finance,Australia,2016
3,1210588,Beginner to Pro - Financial Analysis in Excel ...,https://www.udemy.com/complete-excel-finance-c...,True,95,2451,11,36,All Levels,3.0,2017-05-30 20:07:24+00:00,Business Finance,Russia,2017
4,1011058,How To Maximize Your Profits Trading Options,https://www.udemy.com/how-to-maximize-your-pro...,True,200,1276,45,26,Intermediate Level,2.0,2016-12-13 14:57:18+00:00,Business Finance,South Korea,2016


In [ ]:
df = df.drop([93,95,847,970,2066],axis=0).reset_index(drop=True)

In [ ]:
df['published_timestamp'] = pd.to_datetime(df['published_timestamp'])
df['year'] = df['published_timestamp'].dt.year

In [ ]:
pip install --upgrade panel hvplot pandas

In [ ]:
!pip install plotly

In [ ]:
import plotly.express as px

In [ ]:
# Define a function to update the grouped bar plot based on the selected year
def update_subject(selected_year):
    filtered_df = df[df['year'] == selected_year]
    plot = filtered_df.groupby('subject').size().hvplot.bar(
        stacked=True,
        title=f'Count of Subjects {selected_year}',
        xlabel='subjects',
        ylabel='Count',
        height=300
    )
    return plot

# Define the years available in your dataset
years = sorted(df['year'].unique(), reverse=True)  # Use unique years from your dataset

# Create Select widget for choosing the year
select_year = pn.widgets.Select(name='Select Year', options=list(years))

# Define callback function to update the subscription plot based on the selected year
@pn.depends(select_year.param.value)
def update_subject_callback(selected_year):
    filtered_df = df[df['year'] == selected_year]

    if filtered_df.empty:

        plot = pn.pane.Markdown(f"No data available for {selected_year}")
    else:
        plot = filtered_df.groupby('subject').size().hvplot.bar(
            stacked=True,
            title=f'Count of Subscriptions in {selected_year}',
            xlabel='subject',
            ylabel='Count',
            height=300
        )
    return plot

# Group the data by country and calculate the total number of users and total revenue from each country
country_data = df.groupby(['country', 'year']).agg({'num_subscribers': 'sum', 'price': 'sum'}).reset_index()

# choropleth map showing the number of Netflix users in each country
@pn.depends(select_year.param.value)
def update_user_geo_plot(selected_year):
    filtered_data = country_data[country_data['year'] == selected_year]
    user_geo_plot = px.choropleth(filtered_data, locations='country', locationmode='country names',
                                  color='num_subscribers', title=f'Number of Subject Users by Country in {selected_year}',
                                  hover_name='country', color_continuous_scale='Viridis',
                                  projection='natural earth', scope='world',
                                  center={'lat': 0, 'lon': 0}, range_color=(0, filtered_data['num_subscribers'].max()))
    return user_geo_plot

@pn.depends(select_year.param.value)
def update_revenue_geo_plot(selected_year):
    filtered_data = country_data[country_data['year'] == selected_year]
    revenue_geo_plot = px.choropleth(filtered_data, locations='country', locationmode='country names',
                                     color='price', title=f'Total price paid by users from different country in {selected_year} ($)',
                                     hover_name='country', color_continuous_scale='Viridis',
                                     projection='natural earth', scope='world',
                                     center={'lat': 0, 'lon': 0}, range_color=(0, filtered_data['price'].max()))
    return revenue_geo_plot

# widget for choosing the year in the sidebar
select_year_geo_analysis = pn.widgets.Select(name='Select Year', options=list(years))

# tabs layout including select_year in Geo Analysis tabs
dashboard_tabs = pn.Tabs(
    ('Subjects', pn.Row(update_subject_callback)),
    ('Subscribers by country', pn.Row(update_user_geo_plot)),
    ('Price paid by users and their country', pn.Row(update_revenue_geo_plot))
)



template = pn.template.FastListTemplate(
    title="Udemy User Dashboard",
    sidebar=[
        pn.pane.Markdown("## Udemy User Dashboard"),
        pn.Column(
            select_year,
        )
    ],
    main=[
        pn.Row(dashboard_tabs, width=950)
    ]
)


template.show()

Launching server at http://localhost:36389
